In [4]:
import hopsworks
import pandas as pd

# =========================================================
# 1) Login
# =========================================================
project = hopsworks.login(
    host="eu-west.cloud.hopsworks.ai",
    project="London_traffic"
)
fs = project.get_feature_store()

# =========================================================
# 2) Load existing Feature Groups
# =========================================================
traffic_fg = fs.get_feature_group(name="traffic_temporal_fg", version=1)
labels_fg  = fs.get_feature_group(name="labels_speed_ratio_fg", version=1)
weather_fg = fs.get_feature_group(name="weather_hourly_fg", version=1)
tfl_fg     = fs.get_feature_group(name="tfl_disruptions_hourly_fg", version=1)

# =========================================================
# 3) Helper: expand hourly timestamps to 10-min grid
# =========================================================
def expand_hourly_to_10m(df: pd.DataFrame, time_col: str) -> pd.DataFrame:
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col], utc=True, errors="coerce")
    df = df.dropna(subset=[time_col])

    base = df[time_col].dt.floor("H")
    offsets = pd.to_timedelta([0, 10, 20, 30, 40, 50], unit="m")

    expanded = []
    for off in offsets:
        tmp = df.copy()
        tmp["ts_10m"] = base + off
        expanded.append(tmp)

    out = pd.concat(expanded, ignore_index=True)

    # keep original hourly time col too if you want, but it's optional
    # out = out.drop(columns=[time_col])

    return out

# =========================================================
# 4) Create WEATHER 10-min Feature Group (from hourly)
# =========================================================
weather_df = weather_fg.read(read_options={"use_hive": True})
weather_10m_df = expand_hourly_to_10m(weather_df, time_col="weather_time_utc")

# Select columns (avoid weird duplicates)
weather_10m_cols = ["point_id", "ts_10m", "precipitation", "rain", "snowfall", "temperature_2m"]
weather_10m_df = weather_10m_df[weather_10m_cols].drop_duplicates(subset=["point_id", "ts_10m"])

weather_10m_fg = fs.create_feature_group(
    name="weather_10m_fg",
    version=1,
    primary_key=["point_id", "ts_10m"],
    event_time="ts_10m",
    description="Weather expanded from hourly to 10-min grid for joining with traffic_temporal_fg on (point_id, ts_10m).",
    online_enabled=False
)

weather_10m_fg.insert(weather_10m_df, write_options={"wait_for_job": True})

# =========================================================
# 5) Create TFL 10-min Feature Group (from hourly)
# =========================================================
tfl_df = tfl_fg.read(read_options={"use_hive": True})
tfl_10m_df = expand_hourly_to_10m(tfl_df, time_col="tfl_time_utc")

tfl_10m_cols = ["point_id", "ts_10m", "disruption_count", "is_active", "is_incident", "is_works", "max_ordinal"]
tfl_10m_df = tfl_10m_df[tfl_10m_cols].drop_duplicates(subset=["point_id", "ts_10m"])

tfl_10m_fg = fs.create_feature_group(
    name="tfl_disruptions_10m_fg",
    version=1,
    primary_key=["point_id", "ts_10m"],
    event_time="ts_10m",
    description="TfL disruptions expanded from hourly to 10-min grid for joining with traffic_temporal_fg on (point_id, ts_10m).",
    online_enabled=False
)

tfl_10m_fg.insert(tfl_10m_df, write_options={"wait_for_job": True})

# =========================================================
# 6) Create Feature View using ALL feature groups (10-min join)
# =========================================================
JOIN_KEYS = ["point_id", "ts_10m"]
LABEL_COLS = ["label_speed_ratio_t_plus_30", "label_speed_ratio_t_plus_60"]

q = (
    traffic_fg.select_all()
    .join(labels_fg.select(JOIN_KEYS + LABEL_COLS), on=JOIN_KEYS)
    .join(weather_10m_fg.select_all(), on=JOIN_KEYS)
    .join(tfl_10m_fg.select_all(), on=JOIN_KEYS)
)

fv = fs.create_feature_view(
    name="traffic_speed_ratio_fv",
    version=1,
    query=q,
    labels=LABEL_COLS,
    description="FV for training: traffic (10-min) + labels (+30,+60) + weather(10-min expanded) + tfl(10-min expanded)."
)

print("Created feature view:", fv.name, fv.version)


2026-01-11 13:54:53,567 INFO: Closing external client and cleaning up certificates.
2026-01-11 13:54:53,596 INFO: Connection closed.
2026-01-11 13:54:53,602 INFO: Initializing external client
2026-01-11 13:54:53,603 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-11 13:54:54,573 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3209
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.51s) 


Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/3209/fs/3154/fg/3409


Uploading Dataframe: 100.00% |██████████| Rows 230400/230400 | Elapsed Time: 00:09 | Remaining Time: 00:00


Launching job: weather_10m_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3209/jobs/named/weather_10m_fg_1_offline_fg_materialization/executions
2026-01-11 13:55:22,893 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-11 13:55:26,059 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-11 13:58:03,138 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2026-01-11 13:58:06,307 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-11 13:58:06,440 INFO: Waiting for log aggregation to finish.
2026-01-11 13:58:15,240 INFO: Execution finished successfully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.68s) 


Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/3209/fs/3154/fg/3410


Uploading Dataframe: 100.00% |██████████| Rows 387870/387870 | Elapsed Time: 00:17 | Remaining Time: 00:00


Launching job: tfl_disruptions_10m_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3209/jobs/named/tfl_disruptions_10m_fg_1_offline_fg_materialization/executions
2026-01-11 13:58:49,009 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-11 13:58:52,182 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-11 14:01:30,365 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-11 14:01:30,504 INFO: Waiting for log aggregation to finish.
2026-01-11 14:01:39,326 INFO: Execution finished successfully.
2026-01-11 14:01:39,344 WARNING: Ambiguous features detected during query construction.The feature `point_id` is present in feature groups ['labels_speed_ratio_fg version 1', 'tfl_disruptions_10m_fg version 1', 'traffic_temporal_fg version 1', 'weather_10m_fg version 1']. The fe